<h2>Commentary Analysis - All videos<h2>

In [ ]:
# To customize depending on the user:
path = "C:\\Users\\john.doe\\Documents\\My_Directory"

In [ ]:
#Import libraries

import time
import itertools
import numpy as np
import pandas as pd

#Libraries for webscrapping
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Libraries for vectorisation and clustering
from collections import Counter

#Libraries for preprocessing
from gensim.parsing.preprocessing import remove_stopwords
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
#nltk.download('punkt')

#Libraries for vectorisation and clustering
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

#Libraries for LDA
from gensim import corpora, models

#Libraries for visualization
import webcolors
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px

# Other functions
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        for item in element:
                flat_list.append(item)
    return flat_list

<h3>Retrieve the video comments<h3>

In [ ]:
df = pd.read_csv(path + 'VideoJanuary2022URLs.csv')

In [ ]:
#df = df[1:14]
#df

In [ ]:
#Using Selenium to extract the webpage contents, in particular the comments and associated likes.

# One variable will contains all the video comments as one single list of comments : Comments
# One variable will contains all the video comments as a list-of-list-of-comments-per-video : CommentsSep

Comments = []
CommentsSep = []
Likes = []
for url_ in df['URLs']:
    print(url_)
    CommentsSep_temp = []
    with Chrome() as driver:
        wait = WebDriverWait(driver,10)
        driver.get(url_)
        for item in range(3): 
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            time.sleep(3)
        for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#comment #content-text"))):
            temp = comment.text.strip('\n')
            Comments.append(temp)
            CommentsSep_temp.append(temp)
        for likes in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#vote-count-middle"))):
            Likes.append(likes.text)
    CommentsSep.append(CommentsSep_temp)
        

<h3>LDA topic extraction visualisation<h3>

In [ ]:
CommentsSepTWO = CommentsSep[0:12]
list_of_list_of_tokens = []

for c_ in range(len(CommentsSepTWO)):
    temp = ','.join(CommentsSepTWO[c_])
    temp = test.tokenize(temp)
    temp = [x.lower() for x in temp]
    temp = [lemmatizer.lemmatize(x) for x in temp]
    temp = [word for word in temp if word not in fr_stop]
    temp = [word for word in temp if word not in SW]
    temp = [i for i in temp if not i.isdigit()]
    temp = list(filter(lambda x: len(x) > 1, temp))
    list_of_list_of_tokens.append(temp)

In [ ]:
num_topics = 20

dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
#dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                            id2word=dictionary_LDA, \
                            passes=4, alpha=[0.01] * num_topics, \
                            eta=[0.01] * len(dictionary_LDA.keys()))

In [ ]:
lda_model

In [ ]:
%matplotlib inline 
import pyLDAvis
import pyLDAvis.gensim_models
from gensim import corpora, models
#vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
#pyLDAvis.enable_notebook()
#pyLDAvis.display(vis)